## Importação das bibliotecas principais

In [154]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

## Carregamento dos dados e ajustes

Aqui estamos carregando os dados do dataset `sao-paulo-properties-april-2019.csv` disponível no link https://www.kaggle.com/datasets/argonalyst/sao-paulo-real-estate-sale-rent-april-2019

Após carregar estamos separando o dataset entre as colunas que vamos utilizar para predição (x) e a coluna que será a resposta (y)

Note que foram removidas as duas últimas colunas `latitude` e `logitude`, já que estavam causando muito ruído no treinamento e impactando negativamente nos resultados

In [155]:
df = pd.read_csv("./sao-paulo-properties-april-2019.csv")
# Remove latitude and logitude values since they were causing much noise
x = df.iloc[:,1:df.columns.size-2]
y = df.iloc[:,0]

## Separação das colunas

Esse dataset possui dados numéricos e categóricos. Nessa etapa estamos definindo quais colunas possuem dados numéricos e quais possuem os dados categóricos, pois eles precisam ser pre-processados de maneira diferente

In [156]:
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(x)
categorical_columns = categorical_columns_selector(x)

['Condo',
 'Size',
 'Rooms',
 'Toilets',
 'Suites',
 'Parking',
 'Elevator',
 'Furnished',
 'Swimming Pool',
 'New']

## Criação do preprocessados

Após fazer a separação entre dados categóricos e dados numéricos, vamos definir os pre-processadores para cada um desses dados

Para os dados categóricos usamos o `OneHotEncoder` e para os dados numéricos utilizamos o `StandardScaler` e colocamos numa função `ColumnTransformer` para que faça a transformação dessas colunas na faze de preprocessamento da pipeline que vamos definir adiante

In [157]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
  [
    ("one-hot-encoder", categorical_preprocessor, categorical_columns),
    ("standard_scaler", numerical_preprocessor, numerical_columns),
  ]
)
preprocessor

ColumnTransformer(transformers=[('one-hot-encoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['District', 'Negotiation Type',
                                  'Property Type']),
                                ('standard_scaler', StandardScaler(),
                                 ['Condo', 'Size', 'Rooms', 'Toilets', 'Suites',
                                  'Parking', 'Elevator', 'Furnished',
                                  'Swimming Pool', 'New'])])

## Definição dos algoritmos e pipelines

Aqui selecionamos três algoritmos para fazer os testes com a base de dados

* Multi Layer Processor Regressor
* Linear Regression
* Random Forest Regressor

Esses algoritmos foram selecionados baseados na literatura de apoio que pode ser consultada nas referências

Para cada um desses algoritmos definimos uma pipeline que inclui o pre-processador que definimos na etapa anterior, e tem como objetivo colocar os dados num formato mais adequado para o teinamento, e o algoritmo de regressão para ser processado logo após.

In [158]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

mlp = MLPRegressor(hidden_layer_sizes=(100, 100, 100), activation='relu', early_stopping=True, max_iter=1000)
mlp_pipe = Pipeline(steps=[('pre',preprocessor),('mlpc', mlp)])

line_reg = LinearRegression()
line_reg_pipe = Pipeline(steps=[('pre', preprocessor),('linear_reg', line_reg)])

rf_reg = RandomForestRegressor()
rf_reg_pipe = Pipeline(steps=[('pre', preprocessor), ('randomforest', rf_reg)])


## Treinamento e coleta de métricas

Aqui vamos utilizar a técnica de `k_fold` que consiste em dividir a base de dados em sessões (folds) e treinar de forma individual, dando uma maior distribuição no treinamento dos dados e permitindo identificar conjuntos que possuem um melhor resultado.

Definimos 5 repetições para que possamos identificar como esses treinamentos acontecem e sua consistencia a depender dos dados aleatoriamente selecionados em cada processo de fold.

Então para cada algoritmo vamos treinar e coletar as métricas escolhidas

* R2 Score
* RMSE- root mean squared error

In [159]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
import time

np.random.seed(seed=None)
##
## To evaluate the performance of the models, several error metrics (mean absolute error—MAE, mean square error—MSE, root mean squared error—RMSE) and goodness-of-fit (R² score) were used. This training phase was repeated for each combination of hyperparameters created in the following optimization phase.

repeat = 1
folds = 10
mlp_r2 = []
mlp_rmse =[]
mlp_times = []
linear_r2 = []
linear_rmse = []
linear_times = []
rf_r2 = []
rf_rmse = []
rf_times = []

for i in range(repeat):
    kf = KFold(n_splits=folds)

    start_time = time.time()

    # Treinamento da MLP
    for k, (train, test) in enumerate(kf.split(x)):
        x_train, x_test = x.iloc[train], x.iloc[test]
        y_train, y_test = y.iloc[train], y.iloc[test]

        mlp_pipe.fit(x_train, y_train)
        y_pred = mlp_pipe.predict(x_test)
        r2 = r2_score(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred)

        mlp_r2.append(r2)
        mlp_rmse.append(rmse)

    end_time = time.time()
    mlp_times.append(end_time - start_time)

    start_time = time.time()

    # Treinamento da Regressao Linear
    for k, (train, test) in enumerate(kf.split(x)):
        x_train, x_test = x.iloc[train], x.iloc[test]
        y_train, y_test = y.iloc[train], y.iloc[test]

        line_reg_pipe.fit(x_train, y_train)
        y_pred = line_reg_pipe.predict(x_test)
        r2 = r2_score(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred)

        linear_r2.append(r2)
        linear_rmse.append(rmse)


    end_time = time.time()
    linear_times.append(end_time - start_time)

    start_time = time.time()

    # Treinamento da Random Forest
    for k, (train, test) in enumerate(kf.split(x)):
        x_train, x_test = x.iloc[train], x.iloc[test]
        y_train, y_test = y.iloc[train], y.iloc[test]

        rf_reg_pipe.fit(x_train, y_train)
        y_pred = rf_reg_pipe.predict(x_test)
        r2 = r2_score(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred)

        rf_r2.append(r2)
        rf_rmse.append(rmse)


    end_time = time.time()
    rf_times.append(end_time - start_time)

In [160]:
mlp_r2

[-4.9265980088880355,
 -2.5457608486277907,
 -94.09809161491499,
 0.9011986384891914,
 0.693362723221961,
 0.8921071196557956,
 0.8675885342881446,
 -186.32624184844647,
 0.904108461947383,
 0.8540450071072262]